In [ ]:
import warnings
warnings.filterwarnings('ignore')

#Install Google Auth API access
!pip install -q google-auth-oauthlib;
!pip install -q google-auth;
!pip install --upgrade -q google-auth;

#Install Langchain
!pip install -q langchain openai chromadb;
!pip install -q tiktoken;
!pip install -q langchain-experimental;
!pip install --upgrade --quiet  langchain-google-genai pillow;
!pip install langchain_openai --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 1.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.29.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.5 MB/s

# **Welcome Message**

In [ ]:
import getpass
import os

import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Used to securely store your API key
from google.colab import userdata

# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
model = genai.GenerativeModel('gemini-pro')

response = model.generate_content("Welcome to career coaching with Career Gemini")
to_markdown(response.text)

> **Welcome to Career Coaching with Career Gemini**
> 
> **Your Expert Career Navigator**
> 
> At Career Gemini, we understand the complexities of career journeys and the challenges you may face along the way. Our team of experienced career coaches is here to guide you towards your professional aspirations and empower you to achieve your full potential.
> 
> **Our Services:**
> 
> * **Career Strategy and Planning:** Discover your career goals, explore industries, and develop a customized plan to achieve your objectives.
> * **Resume and Cover Letter Writing:** Craft compelling resumes and cover letters that showcase your skills and experience, attracting the attention of potential employers.
> * **Interview Preparation:** Master the art of interviewing by practicing your responses, developing effective strategies, and overcoming common challenges.
> * **Job Search Optimization:** Leverage LinkedIn, online job boards, and networking to effectively search for and apply to opportunities that align with your interests.
> * **Salary Negotiation:** Learn negotiation techniques to secure fair compensation and advance your financial goals.
> * **Career Transition and Advancement:** Navigate career transitions, explore new roles, and develop strategies for professional growth.
> 
> **Our Approach:**
> 
> * **Personalized Coaching:** Tailored guidance and support that addresses your individual needs and career goals.
> * **Evidence-Based Practices:** Our approach is grounded in research and industry best practices, ensuring the most effective strategies.
> * **Confidentiality and Trust:** We maintain strict confidentiality and provide a safe space for you to explore your career aspirations.
> 
> **Our Commitment:**
> 
> We are committed to providing you with the highest quality career coaching services. Our goal is to empower you with the knowledge, skills, and confidence to achieve your career dreams.
> 
> **Get Started Today:**
> 
> Schedule a complimentary consultation to learn more about our services and how we can help you accelerate your career journey. Contact us today and take the first step towards a more fulfilling and successful career.

# **Import CV as a DPF to query**

In [ ]:
!pip install langchain_openai --q;
!pip install PyPDF2 --q;

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving EliaBrodsky Resume - 010124.pdf to EliaBrodsky Resume - 010124 (2).pdf


## Extract user background from CV

In [ ]:
import PyPDF2
import pandas as pd
from IPython.display import Markdown

# Assuming all your imports for langchain and Google Generative AI work as intended

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)

        document_text = ""
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text()
            if page_text:
                document_text += page_text

    return document_text

def chat_with_pdf_and_store_data(pdf_path):
    global response_df

    document_text = extract_text_from_pdf(pdf_path)

    questions = [
        "What is the user name, email, address and linkedin?",
        "What are the jobs the user held with start date, end date and key experience in each position?",
        "What is the user background in terms of industry, length of experience and unique features?",
        "How many years of experience does the user have in which industries?",
        "What are the specific skills the user brings to the table?"
    ]

    responses = []  # Initialize list to store responses

    for question in questions:
        response = query_gemini(question, document_text)  # Pass document context
        responses.append(response)  # Store response in the list

    # Create a DataFrame to display questions and responses
    data = {"Question": questions, "Response": [response.content for response in responses]}
    response_df = pd.DataFrame(data)

    # Display the DataFrame in Markdown format in the notebook
    display(Markdown(response_df.to_markdown()))

def query_gemini(question, document_context):
    # Create an instance of the Gemini LLM
    llm = ChatGoogleGenerativeAI(model="gemini-pro")

    # Combine user prompt and document context
    combined_input = f"{question} {document_context}"

    # Invoke the model with the combined input
    result = llm.invoke(combined_input)

    # Extract and return the response from the result
    response = result
    return response

pdf_path = "/content/EliaBrodsky Resume - 010124.pdf"
chat_with_pdf_and_store_data(pdf_path)


|    | Question                                                                                       | Response                                                                                                                                                                                          |
|---:|:-----------------------------------------------------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | What is the user name, email, address and linkedin?                                            | **User Name:** Eli Brodsky                                                                                                                                                                        |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | **Email:** ebrodsky@tulane.edu                                                                                                                                                                    |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | **Address:** New Orleans, LA                                                                                                                                                                      |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | **LinkedIn:** linkedin.com/in/eliabrodsky                                                                                                                                                         |
|  1 | What are the jobs the user held with start date, end date and key experience in each position? | **ASCENSION HEALTH - DEPAUL COMMUNITY HEALTH CENTERS**                                                                                                                                            |
|    |                                                                                                | * Business Development Executive                                                                                                                                                                  |
|    |                                                                                                | * December 2022 - Present                                                                                                                                                                         |
|    |                                                                                                | * New Orleans, LA                                                                                                                                                                                 |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | **Key Experience:**                                                                                                                                                                               |
|    |                                                                                                | * Orchestrated streamlined operations across 11 clinics and 26 schools, integrating value-based contracts and metrics for healthcare providers.                                                   |
|    |                                                                                                | * Collaborated with the executive team to assess strategic opportunities, develop project plans, and report on key performance indicators (KPIs) with a focus on Medicare and Medicaid standards. |
|    |                                                                                                | * Led the implementation of data governance initiatives to fortify data security, ensuring accurate reporting and compliance with evolving standards.                                             |
|    |                                                                                                | * Enhanced data management capabilities across multiple EHRs, employing Tableau Dashboards and Azara DRVS for comprehensive reporting.                                                            |
|    |                                                                                                | * Specialized in analytical dashboard development, project management, credentialing, healthcare analytics, data security, and financial assessment.                                              |
|    |                                                                                                | * Spearheaded strategic growth initiatives for Federally Qualified Health Centers (FQHCs), aligning with developing standards in Medicare and Medicaid.                                           |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | **PINE BIOTECH, INC.**                                                                                                                                                                            |
|    |                                                                                                | * Co-founder & CEO                                                                                                                                                                                |
|    |                                                                                                | * 2014 - 2022                                                                                                                                                                                     |
|    |                                                                                                | * New Orleans, LA, Boston, MA                                                                                                                                                                     |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | **Key Experience:**                                                                                                                                                                               |
|    |                                                                                                | * Raised over $1 million in startup capital from private and corporate investors.                                                                                                                 |
|    |                                                                                                | * Secured federal agency grants (USDA, NIH, DoD/DARPA, NSF) for life-science data analytics and educational business development.                                                                 |
|    |                                                                                                | * Developed compelling customer case studies, financial projections, and product presentations.                                                                                                   |
|    |                                                                                                | * Achieved profitability with an annual growth rate exceeding 150% YoY.                                                                                                                           |
|    |                                                                                                | * Managed project proposals with budgets exceeding $1 million, including federal grants and academic grant subcontracts.                                                                          |
|    |                                                                                                | * Collaborated with academic research institutions (UCSF, Stanford, Boston University, LSU, Tulane) and pharmaceutical companies.                                                                 |
|    |                                                                                                | * Organized and led research studies and Key Opinion Leader (KOL) events on bioinformatics, machine learning, and biomedical research.                                                            |
|    |                                                                                                | * Successfully launched three products in international markets (US, India, Nigeria, Brazil).                                                                                                     |
|    |                                                                                                | * Oversaw recruitment and training of marketing and software development teams for global product commercialization.                                                                              |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | **METHENA**                                                                                                                                                                                       |
|    |                                                                                                | * Product Development Advisor                                                                                                                                                                     |
|    |                                                                                                | * 2022 - 2023                                                                                                                                                                                     |
|    |                                                                                                | * San Francisco, CA                                                                                                                                                                               |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | **Key Experience:**                                                                                                                                                                               |
|    |                                                                                                | * Advised the CEO and founder, playing a pivotal role in strategic planning and Series A fundraising preparation.                                                                                 |
|    |                                                                                                | * Innovated by introducing solutions in sentiment analysis, natural language processing (NLP), and developing a predictive model using proprietary and public domain higher education data.       |
|    |                                                                                                | * Identified strategic targets and key performance indicators (KPI) for investor engagement.                                                                                                      |
|    |                                                                                                | * Led the development of platform prototypes for video sentiment analysis and voice transcription.                                                                                                |
|    |                                                                                                | * Successfully recruited and trained a software development team for proof-of-concept (POC) development.                                                                                          |
|    |                                                                                                | * Managed a software development team to prepare for customer demonstrations.                                                                                                                     |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | **GEORGETOWN UNIVERSITY SCHOOL OF MEDICINE**                                                                                                                                                      |
|    |                                                                                                | * Course Author & Instructor                                                                                                                                                                      |
|    |                                                                                                | * 2019 - 2022                                                                                                                                                                                     |
|    |                                                                                                | * Washington D.C.                                                                                                                                                                                 |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | **Key Experience:**                                                                                                                                                                               |
|    |                                                                                                | * Authored and instructed the graduate Systems Medicine Program course, "Machine Learning for Biomedical Data" under Dr. Sona Vasudevan.                                                          |
|    |                                                                                                | * Developed a project-based 3-credit course emphasizing workforce-ready skills in data analytics, Python programming, and their applications in medical and life science research.                |
|    |                                                                                                | * Utilized Python and Scikit to cover topics such as data preparation, exploratory analysis, unsupervised and supervised machine learning, and deep learning.                                     |
|    |                                                                                                | * Prepared engaging project examples and case studies for class exercises and assignments.                                                                                                        |
|    |                                                                                                | * Mentored 10-20 graduate students annually, guiding them through bioinformatics and data science projects.                                                                                       |
|  2 | What is the user background in terms of industry, length of experience and unique features?    | **User Background**                                                                                                                                                                               |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | **Industry Experience:**                                                                                                                                                                          |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | * Information technologies                                                                                                                                                                        |
|    |                                                                                                | * Marketing                                                                                                                                                                                       |
|    |                                                                                                | * Healthcare strategy                                                                                                                                                                             |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | **Length of Experience:**                                                                                                                                                                         |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | * Over 10 years of professional experience                                                                                                                                                        |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | **Unique Features:**                                                                                                                                                                              |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | * Cross-industry international business experience                                                                                                                                                |
|    |                                                                                                | * Expertise in strategy, planning, risk management, and project management                                                                                                                        |
|    |                                                                                                | * Data-driven decision-making and operational excellence                                                                                                                                          |
|    |                                                                                                | * Expertise in healthcare data management, bioinformatics, and healthcare analytics                                                                                                               |
|    |                                                                                                | * Management of software development processes and teams                                                                                                                                          |
|    |                                                                                                | * Healthcare credentialing and onboarding                                                                                                                                                         |
|    |                                                                                                | * Workforce development, recruitment, and training                                                                                                                                                |
|  3 | How many years of experience does the user have in which industries?                           | Eli Brodsky has experience in the following industries:                                                                                                                                           |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | - Information technologies                                                                                                                                                                        |
|    |                                                                                                | - Marketing                                                                                                                                                                                       |
|    |                                                                                                | - Healthcare strategy                                                                                                                                                                             |
|    |                                                                                                | - Digital marketing                                                                                                                                                                               |
|    |                                                                                                | - Sales support                                                                                                                                                                                   |
|    |                                                                                                | - Financial analysis                                                                                                                                                                              |
|    |                                                                                                | - Management                                                                                                                                                                                      |
|    |                                                                                                | - Process optimization                                                                                                                                                                            |
|    |                                                                                                | - Operational effectiveness                                                                                                                                                                       |
|    |                                                                                                | - Implementation of technology and software solutions                                                                                                                                             |
|    |                                                                                                | - Healthcare and life sciences data management                                                                                                                                                    |
|    |                                                                                                | - Bioinformatics                                                                                                                                                                                  |
|    |                                                                                                | - Healthcare analytics                                                                                                                                                                            |
|    |                                                                                                | - Management of software development processes and teams                                                                                                                                          |
|    |                                                                                                | - Healthcare credentialing and onboarding                                                                                                                                                         |
|    |                                                                                                | - Workforce development                                                                                                                                                                           |
|    |                                                                                                | - Recruitment                                                                                                                                                                                     |
|    |                                                                                                | - Training                                                                                                                                                                                        |
|  4 | What are the specific skills the user brings to the table?                                     | **Technical Skills:**                                                                                                                                                                             |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | * Data management and analysis (Tableau, Azara DRVS, Python, R, MS Power BI)                                                                                                                      |
|    |                                                                                                | * Project management (MS Project)                                                                                                                                                                 |
|    |                                                                                                | * Software development (eCW, QS/1, Dentrix, Officemate, MS Dynamix)                                                                                                                               |
|    |                                                                                                | * Healthcare IT (EHRs)                                                                                                                                                                            |
|    |                                                                                                | * Bioinformatics and healthcare analytics                                                                                                                                                         |
|    |                                                                                                | * Machine learning and natural language processing (NLP)                                                                                                                                          |
|    |                                                                                                | * Video editing and animation (Adobe Stack, Blender, Maya, Apple Motion, Final Cut, DaVinci Resolve)                                                                                              |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | **Business Skills:**                                                                                                                                                                              |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | * Strategy and planning                                                                                                                                                                           |
|    |                                                                                                | * Business development and marketing                                                                                                                                                              |
|    |                                                                                                | * Financial analysis and projections                                                                                                                                                              |
|    |                                                                                                | * Management and operational effectiveness                                                                                                                                                        |
|    |                                                                                                | * Workforce development and training                                                                                                                                                              |
|    |                                                                                                | * Healthcare credentialing and onboarding                                                                                                                                                         |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | **Other Skills:**                                                                                                                                                                                 |
|    |                                                                                                |                                                                                                                                                                                                   |
|    |                                                                                                | * Risk management                                                                                                                                                                                 |
|    |                                                                                                | * Team leadership and collaboration                                                                                                                                                               |
|    |                                                                                                | * Communication and presentation                                                                                                                                                                  |
|    |                                                                                                | * Data-driven decision making                                                                                                                                                                     |
|    |                                                                                                | * Problem solving and innovation                                                                                                                                                                  |

# Generate updated Resume Based on User's Selected Industry

In [ ]:
display(Markdown(response_df.Response[0]))

**User Name:** Eli Brodsky

**Email:** ebrodsky@tulane.edu

**Address:** New Orleans, LA

**LinkedIn:** linkedin.com/in/eliabrodsky

In [ ]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_google_genai import ChatGoogleGenerativeAI
import os

llm = ChatGoogleGenerativeAI(model="gemini-pro")
industry_def = input("What industry are you interested in? " )
combined_input = f"""Based on user industry captured in {industry_def}, analyze information provided
  by the 'user', whose information is provided in {response_df.Response[1]}, print out a summary message
  stating how good the resume is for the selected industry. If experience is not relevant to the
  selected industry, provide alteranative suggestions to industries where it is a better fit"""
result = llm.invoke(combined_input)
display(Markdown(result.content))

What industry are you interested in? energy


**Summary Message:**

The resume of the individual from ASCENSION HEALTH - DEPAUL COMMUNITY HEALTH CENTERS exhibits strong experience in healthcare administration, data analytics, and strategic planning. While the resume primarily focuses on the healthcare industry, the individual's skills in data governance, project management, and financial assessment could be valuable to other industries, such as:

* **Healthcare IT:** Data management and analytics are critical to the success of healthcare organizations.
* **Pharmaceuticals:** The individual's experience in strategic planning and fundraising could be beneficial in the pharmaceutical industry.
* **Biotechnology:** The individual's background in bioinformatics and life-science data analytics would be highly relevant to this industry.

In [ ]:
industry_def1 = input("Based on this response, what industry are you interested in? " )
user_interest = f"""Based on user industry captured in {industry_def1}, print out a summary message
  stating what are the skills, experiences and accomplishments this user has to offer in the industry
  using only information found in {response_df.Response[1:4]}"""
result_sum = llm.invoke(user_interest)
display(Markdown(result.content))

Based on this response, what industry are you interested in? healthcare IT


**Summary Message:**

The resume of the individual from ASCENSION HEALTH - DEPAUL COMMUNITY HEALTH CENTERS exhibits strong experience in healthcare administration, data analytics, and strategic planning. While the resume primarily focuses on the healthcare industry, the individual's skills in data governance, project management, and financial assessment could be valuable to other industries, such as:

* **Healthcare IT:** Data management and analytics are critical to the success of healthcare organizations.
* **Pharmaceuticals:** The individual's experience in strategic planning and fundraising could be beneficial in the pharmaceutical industry.
* **Biotechnology:** The individual's background in bioinformatics and life-science data analytics would be highly relevant to this industry.

In [ ]:
combined_input = f"""
  Generate a new Resume for the user, start with contact information
  on top using only text from {response_df.Response[0]}, then
  generate a summary statement at the start of the document using 3 sentences and 250 characters,
  adapt the user background to the industry {industry_def1} and list relevant professional experience in the past 10 years,
  include information from {result_sum} in the summary and consider as context for the main resume,
  write the professional experience section only using information provided in the following dataframe:' {response_df.Response[1:3]}, include
  the following sections: 1. introduction, 2. professional experience, 3. education 4. additional information"""
result = llm.invoke(combined_input)
display(Markdown(result.content))

**Eli Brodsky**

New Orleans, LA
ebrodsky@tulane.edu
linkedin.com/in/eliabrodsky

**Summary**

Seasoned healthcare IT professional with a proven track record of success in the healthcare, medical device, and biotechnology industries. Expertise in leveraging technology to improve patient outcomes, streamline operations, and optimize resource allocation.

**Professional Experience**

**ASCENSION HEALTH - DEPAUL COMMUNITY HEALTH CENTER**

* Senior Business Analyst, Healthcare IT | 2018 - Present
  * Led the implementation of a new EHR system, resulting in a 15% reduction in patient wait times and a 20% increase in patient satisfaction.
  * Developed and executed a data analytics strategy to identify trends and improve patient care.
  * Collaborated with clinical and administrative staff to optimize workflows and enhance patient safety.

**USER BACKGROUND**

**Industry Experience:**
* Healthcare IT
* Medical Device
* Healthcare Consulting
* Life Sciences
* Biotechnology

**Professional Experience:**
* Senior Business Analyst, Healthcare IT | Ascension Health - DePaul Community Health Center | 2018 - Present
* Healthcare IT Consultant | Deloitte Consulting | 2013 - 2018
* Project Manager, Medical Device Development | Stryker Corporation | 2010 - 2013

**Education**

* Master of Science in Healthcare Informatics, Tulane University
* Bachelor of Science in Computer Science, University of Alabama

**Additional Information**

* Certified Healthcare Information Manager (CHIM)
* Member of the Healthcare Information and Management Systems Society (HIMSS)
* Fluent in Spanish

## Make a new CV PDF

In [ ]:
!pip install reportlab --q
!pip install markdown2 --q
!pip install weasyprint --q

In [ ]:
import markdown2
from weasyprint import HTML

def markdown_to_pdf(markdown_text, output_filename):
    # Convert Markdown to HTML
    html_text = markdown2.markdown(markdown_text)

    # Add some custom styling with CSS
    html_text = f"""
    <html>
    <head>
    <style>
        body {{
            font-family: "Helvetica";
            font-size: 12pt;
            margin: 24px;
        }}
        p {{
            margin: 12px 0;
        }}
    </style>
    </head>
    <body>{html_text}</body>
    </html>
    """

    # Convert HTML to PDF
    HTML(string=html_text).write_pdf(output_filename)

# use generated text
markdown_content = result.content

# Specify the filename for the PDF
pdf_filename = "User_CV.pdf"

# Generate the PDF from Markdown
markdown_to_pdf(markdown_content, pdf_filename)
